In [ ]:
# We are trying to generate Canopy Height Model from the DSM and DTM files using rasterio library

import os
import glob
import numpy as np
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import reproject


# PARAMETERS


elev_th = -0.01

in_dir_dsm = r"C:\............Cover_crop"
in_dir_dtm = r"C:\Users............"

out_dir = r"C:\Users\..........CHM_files"
os.makedirs(out_dir, exist_ok=True)

files_dsm = sorted(glob.glob(os.path.join(in_dir_dsm, "*dsm.tif")))
files_dtm = sorted(glob.glob(os.path.join(in_dir_dtm, "*dtm.tif")))


# PROCESS EACH DSM / DTM PAIR


for i, (dsm_fn, dtm_fn) in enumerate(zip(files_dsm, files_dtm)):

    print(f"Processing ({i+1}/{len(files_dsm)}) "
          f"[{(i+1)/len(files_dsm)*100:.2f}%]")

    # ------------------------------------------------------
    # READ DSM
    # ------------------------------------------------------
    with rasterio.open(dsm_fn) as dsm_ds:
        dsm = dsm_ds.read(1).astype(np.float32)
        dsm_meta = dsm_ds.meta.copy()

    # ------------------------------------------------------
    # READ DTM (AND RESAMPLE TO DSM GRID IF NEEDED)
    # ------------------------------------------------------
    with rasterio.open(dtm_fn) as dtm_ds:

        dtm = np.full(
            (dsm_meta["height"], dsm_meta["width"]),
            np.nan,
            dtype=np.float32
        )

        reproject(
            source=rasterio.band(dtm_ds, 1),
            destination=dtm,
            src_transform=dtm_ds.transform,
            src_crs=dtm_ds.crs,
            dst_transform=dsm_meta["transform"],
            dst_crs=dsm_meta["crs"],
            resampling=Resampling.bilinear
        )

    
    # CHM COMPUTATION
    chm = dsm - dtm

    # Apply elevation threshold and physical constraints
    chm[(dsm < elev_th) | (dtm < elev_th) | (chm < 0)] = np.nan


    # SAVE CHM
    out_basename = os.path.splitext(os.path.basename(dsm_fn))[0][:8]
    out_fn = os.path.join(out_dir, out_basename + "_chm.tif")

    dsm_meta.update({
        "dtype": "float32",
        "count": 1,
        "nodata": np.nan
    })

    with rasterio.open(out_fn, "w", **dsm_meta) as dst:
        dst.write(chm, 1)

    print("Saved:", out_fn)


In [ ]:
#### For sampling the CHM Stats Within plots ##

import os
import glob
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.features import geometry_mask
from shapely.geometry import mapping


# INPUTS


chm_dir = r"C:\Users\..............\CHM_files"
shp_in  = r"C:\Users\.................Plots_Shapefile.shp"

out_dir = r"C:\Users\................Results_raw_sampling"
os.makedirs(out_dir, exist_ok=True)

out_shp = os.path.join(out_dir, "cc_raw_chm_stats.shp")

files_chm = sorted(glob.glob(os.path.join(chm_dir, "*chm*.dat")))


# READ INPUT SHAPEFILE (DO NOT MODIFY)


gdf_in = gpd.read_file(shp_in)

if gdf_in.empty:
    raise ValueError("Input shapefile has no features.")

# Create a COPY for output
gdf_out = gdf_in.copy()


# PROCESS EACH CHM FILE


for chm_path in files_chm:

    base = os.path.basename(chm_path)

    # Extract date (20YYYYMMDD)
    date_str = base.split("20", 1)[1][:6]  # YYMMDD for shapefile 

    print(f"\nProcessing CHM: {base}")

    with rasterio.open(chm_path) as src:

        # Reproject copy ONLY
        if gdf_out.crs != src.crs:
            gdf_proj = gdf_out.to_crs(src.crs)
        else:
            gdf_proj = gdf_out

        pixel_area = abs(src.transform.a * src.transform.e)

        # Create new fields
        gdf_out[f"avCH{date_str}"] = np.nan
        gdf_out[f"mxCH{date_str}"] = np.nan
        gdf_out[f"90CH{date_str}"] = np.nan
        gdf_out[f"95CH{date_str}"] = np.nan
        gdf_out[f"99CH{date_str}"] = np.nan
        gdf_out[f"sdCH{date_str}"] = np.nan
        gdf_out[f"CV{date_str}"]   = np.nan

        # Loop plots
        for idx, geom in enumerate(gdf_proj.geometry):

            # Mask ONLY to locate pixels (no filtering of values)
            mask = geometry_mask(
                [mapping(geom)],
                out_shape=(src.height, src.width),
                transform=src.transform,
                invert=True
            )

            rows, cols = np.where(mask)

            # Convert pixel indices → coordinates
            xs, ys = rasterio.transform.xy(src.transform, rows, cols)

            # Sample raster values
            values = np.array(list(src.sample(zip(xs, ys)))).flatten()

            if values.size == 0:
                continue  # leave NaN

            # RAW statistics (no filtering)
            gdf_out.at[idx, f"avCH{date_str}"] = np.mean(values)
            gdf_out.at[idx, f"mxCH{date_str}"] = np.max(values)
            gdf_out.at[idx, f"90CH{date_str}"] = np.percentile(values, 90)
            gdf_out.at[idx, f"95CH{date_str}"] = np.percentile(values, 95)
            gdf_out.at[idx, f"99CH{date_str}"] = np.percentile(values, 99)
            gdf_out.at[idx, f"sdCH{date_str}"] = np.std(values)
            gdf_out.at[idx, f"CV{date_str}"]   = np.sum(values) * pixel_area


# WRITE NEW SHAPEFILE


gdf_out.to_file(out_shp, driver="ESRI Shapefile")

print("\n Finished")
print(f"New shapefile written to:\n{out_shp}")
